In [1]:
from PIL import Image
from io import BytesIO
import os, sys, requests, json, random, math, imageio, shutil, glob, requests
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.patches as mpatches
from skimage.filters import threshold_otsu
from skimage.io import imread, imsave
from skimage.color import rgb2gray
from skimage.measure import label, regionprops
import imgaug as ia
from imgaug import augmenters as iaa
from imgaug.augmentables.bbs import BoundingBox, BoundingBoxesOnImage
sys.path.append('/data/users/g6114450078/PyTorch-YOLOv3/preprocessing')
from aug import *

In [2]:
def reset(percent=20):
    return random.randrange(100) < percent

In [3]:
# Training and Validate Sets
degs = [45,90,135,180,225,270,315]
# degs = [90,180,270,360,450,540,630]
img_bbs = []
count = 1
phase_count = {
    "train": 1,
    "test": 1
}

labels = os.path.join('training.json')
with open(labels,'r') as json_file:
    datas = json.load(json_file)
    for data in datas:
        read_path = (data['image']['pathname'])[1:]
        print(read_path)
        phase = "train" if (count % 2) == 0 else "test"            
        img_name_new = "pv-" + phase + "-{:010d}".format(phase_count[phase])
        print(img_name_new)
        save_path = os.path.join('../data/custom/images',img_name_new+".jpg")
        img = Image.open(read_path)
        objects = data['objects']
        rects = []
        result = ""
        img = np.array(img)
        h,w,c = img.shape
        RBCs_flag = reset(100)
        for obj in objects:
            minr = obj['bounding_box']['minimum']['r']
            minc = obj['bounding_box']['minimum']['c']
            maxr = obj['bounding_box']['maximum']['r']
            maxc = obj['bounding_box']['maximum']['c']
            x1 = minc / w
            y1 = minr / h
            bw = (maxc - minc) / w
            bh = (maxr - minr) / h
            xc = x1 + (bw / 2)
            yc = y1 + (bh / 2)
            category = obj['category']
            if (category == "red blood cell") and (RBCs_flag or phase == "test"):
                result += "\n"+ "0 " + str(xc) + " " + str(yc) + " " + str(bw) + " " + str(bh)
                rects.append(BoundingBox(x1=minc,y1=minr,x2=maxc,y2=maxr,label=0))
            elif category == "ring" or category == "trophozoite" or category == "schizont" or category == "gametocyte":
                result += "\n" + "1 " + str(xc) + " " + str(yc) + " " + str(bw) + " " + str(bh)
                rects.append(BoundingBox(x1=minc,y1=minr,x2=maxc,y2=maxr,label=1))
        result = result[1:]
        if len(result) == 0:
            count += 1
            phase_count[phase] += 1
            continue
        Image.fromarray(img).save(save_path)
        f = open(os.path.join('../data/custom/labels',img_name_new+".txt"),'w')
        f.write(result)
        f.close()
        bbs = BoundingBoxesOnImage(rects,shape=img.shape)
        count += 1
        phase_count[phase] += 1
        if phase == "train":
            phase_count = rotate(img=img,bbs=bbs,specie="pv",offset=10,phase_count=phase_count)
        
            aug_hor, bbs_hor, phase_count = flipHor(img=img,bbs=bbs,specie="pv",phase_count=phase_count)
            phase_count = rotate(img=aug_hor,bbs=bbs_hor,specie="pv",offset=10,phase_count=phase_count)
        
            aug_ver, bbs_ver, phase_count = flipVer(img=aug_hor,bbs=bbs_hor,specie="pv",phase_count=phase_count)
            phase_count = rotate(img=aug_ver,bbs=bbs_ver,specie="pv",offset=10,phase_count=phase_count)
        
            aug_ver2, bbs_ver2, phase_count = flipVer(img=img,bbs=bbs,specie="pv",phase_count=phase_count)
            phase_count = rotate(img=aug_ver2,bbs=bbs_ver2,specie="pv",offset=10,phase_count=phase_count)

images/8d02117d-6c71-4e47-b50a-6cc8d5eb1d55.png
pv-test-0000000001
images/10be6380-cbbb-4886-8b9e-ff56b1710576.png
pv-train-0000000001
2 45 pv-train-0000000002
3 90 pv-train-0000000003
4 135 pv-train-0000000004
5 180 pv-train-0000000005
6 225 pv-train-0000000006
7 270 pv-train-0000000007
8 315 pv-train-0000000008
9 flipLR pv-train-0000000009
10 45 pv-train-0000000010
11 90 pv-train-0000000011
12 135 pv-train-0000000012
13 180 pv-train-0000000013
14 225 pv-train-0000000014
15 270 pv-train-0000000015
16 315 pv-train-0000000016
17 flipUD pv-train-0000000017
18 45 pv-train-0000000018
19 90 pv-train-0000000019
20 135 pv-train-0000000020
21 180 pv-train-0000000021
22 225 pv-train-0000000022
23 270 pv-train-0000000023
24 315 pv-train-0000000024
25 flipUD pv-train-0000000025
26 45 pv-train-0000000026
27 90 pv-train-0000000027
28 135 pv-train-0000000028
29 180 pv-train-0000000029
30 225 pv-train-0000000030
31 270 pv-train-0000000031
32 315 pv-train-0000000032
images/6b14c855-8561-417c-97a4-63fa

263 270 pv-train-0000000263
264 315 pv-train-0000000264
265 flipLR pv-train-0000000265
266 45 pv-train-0000000266
267 90 pv-train-0000000267
268 135 pv-train-0000000268
269 180 pv-train-0000000269
270 225 pv-train-0000000270
271 270 pv-train-0000000271
272 315 pv-train-0000000272
273 flipUD pv-train-0000000273
274 45 pv-train-0000000274
275 90 pv-train-0000000275
276 135 pv-train-0000000276
277 180 pv-train-0000000277
278 225 pv-train-0000000278
279 270 pv-train-0000000279
280 315 pv-train-0000000280
281 flipUD pv-train-0000000281
282 45 pv-train-0000000282
283 90 pv-train-0000000283
284 135 pv-train-0000000284
285 180 pv-train-0000000285
286 225 pv-train-0000000286
287 270 pv-train-0000000287
288 315 pv-train-0000000288
images/a8cb75ba-3f2f-40d8-8bc7-c79ebd1b9603.png
pv-test-0000000010
images/63d36d2c-43a9-49ac-9504-028b4968c860.png
pv-train-0000000289
290 45 pv-train-0000000290
291 90 pv-train-0000000291
292 135 pv-train-0000000292
293 180 pv-train-0000000293
294 225 pv-train-0000000

526 225 pv-train-0000000526
527 270 pv-train-0000000527
528 315 pv-train-0000000528
529 flipUD pv-train-0000000529
530 45 pv-train-0000000530
531 90 pv-train-0000000531
532 135 pv-train-0000000532
533 180 pv-train-0000000533
534 225 pv-train-0000000534
535 270 pv-train-0000000535
536 315 pv-train-0000000536
537 flipUD pv-train-0000000537
538 45 pv-train-0000000538
539 90 pv-train-0000000539
540 135 pv-train-0000000540
541 180 pv-train-0000000541
542 225 pv-train-0000000542
543 270 pv-train-0000000543
544 315 pv-train-0000000544
images/5bf91ff9-80df-49c0-9c3e-599357d95530.png
pv-test-0000000018
images/fe851c88-692d-4199-87e0-d19d9c4eb591.png
pv-train-0000000545
546 45 pv-train-0000000546
547 90 pv-train-0000000547
548 135 pv-train-0000000548
549 180 pv-train-0000000549
550 225 pv-train-0000000550
551 270 pv-train-0000000551
552 315 pv-train-0000000552
553 flipLR pv-train-0000000553
554 45 pv-train-0000000554
555 90 pv-train-0000000555
556 135 pv-train-0000000556
557 180 pv-train-0000000

790 225 pv-train-0000000790
791 270 pv-train-0000000791
792 315 pv-train-0000000792
793 flipUD pv-train-0000000793
794 45 pv-train-0000000794
795 90 pv-train-0000000795
796 135 pv-train-0000000796
797 180 pv-train-0000000797
798 225 pv-train-0000000798
799 270 pv-train-0000000799
800 315 pv-train-0000000800
images/fd1302e2-3cd2-4a1a-9c13-00f810a9c41b.png
pv-test-0000000026
images/ad36bd67-594b-4f63-93e3-886af0bc618e.png
pv-train-0000000801
802 45 pv-train-0000000802
803 90 pv-train-0000000803
804 135 pv-train-0000000804
805 180 pv-train-0000000805
806 225 pv-train-0000000806
807 270 pv-train-0000000807
808 315 pv-train-0000000808
809 flipLR pv-train-0000000809
810 45 pv-train-0000000810
811 90 pv-train-0000000811
812 135 pv-train-0000000812
813 180 pv-train-0000000813
814 225 pv-train-0000000814
815 270 pv-train-0000000815
816 315 pv-train-0000000816
817 flipUD pv-train-0000000817
818 45 pv-train-0000000818
819 90 pv-train-0000000819
820 135 pv-train-0000000820
821 180 pv-train-0000000

1050 45 pv-train-0000001050
1051 90 pv-train-0000001051
1052 135 pv-train-0000001052
1053 180 pv-train-0000001053
1054 225 pv-train-0000001054
1055 270 pv-train-0000001055
1056 315 pv-train-0000001056
images/21ea4841-8998-4c04-b345-bc99cd8ecee6.png
pv-test-0000000034
images/cf90d82f-2814-4fb7-a3ff-a86b7c5efca7.png
pv-train-0000001057
1058 45 pv-train-0000001058
1059 90 pv-train-0000001059
1060 135 pv-train-0000001060
1061 180 pv-train-0000001061
1062 225 pv-train-0000001062
1063 270 pv-train-0000001063
1064 315 pv-train-0000001064
1065 flipLR pv-train-0000001065
1066 45 pv-train-0000001066
1067 90 pv-train-0000001067
1068 135 pv-train-0000001068
1069 180 pv-train-0000001069
1070 225 pv-train-0000001070
1071 270 pv-train-0000001071
1072 315 pv-train-0000001072
1073 flipUD pv-train-0000001073
1074 45 pv-train-0000001074
1075 90 pv-train-0000001075
1076 135 pv-train-0000001076
1077 180 pv-train-0000001077
1078 225 pv-train-0000001078
1079 270 pv-train-0000001079
1080 315 pv-train-00000010

...

18155 90 pv-train-0000018155
18156 135 pv-train-0000018156
18157 180 pv-train-0000018157
18158 225 pv-train-0000018158
18159 270 pv-train-0000018159
18160 315 pv-train-0000018160
18161 flipUD pv-train-0000018161
18162 45 pv-train-0000018162
18163 90 pv-train-0000018163
18164 135 pv-train-0000018164
18165 180 pv-train-0000018165
18166 225 pv-train-0000018166
18167 270 pv-train-0000018167
18168 315 pv-train-0000018168
18169 flipUD pv-train-0000018169
18170 45 pv-train-0000018170
18171 90 pv-train-0000018171
18172 135 pv-train-0000018172
18173 180 pv-train-0000018173
18174 225 pv-train-0000018174
18175 270 pv-train-0000018175
18176 315 pv-train-0000018176
images/2afe2750-c963-46f9-a1ad-9260e80679ca.png
pv-test-0000000569
images/9ec1dcec-87c2-4aca-8e6b-0db3cd05509c.png
pv-train-0000018177
18178 45 pv-train-0000018178
18179 90 pv-train-0000018179
18180 135 pv-train-0000018180
18181 180 pv-train-0000018181
18182 225 pv-train-0000018182
18183 270 pv-train-0000018183
18184 315 pv-train-0000018

18403 90 pv-train-0000018403
18404 135 pv-train-0000018404
18405 180 pv-train-0000018405
18406 225 pv-train-0000018406
18407 270 pv-train-0000018407
18408 315 pv-train-0000018408
18409 flipLR pv-train-0000018409
18410 45 pv-train-0000018410
18411 90 pv-train-0000018411
18412 135 pv-train-0000018412
18413 180 pv-train-0000018413
18414 225 pv-train-0000018414
18415 270 pv-train-0000018415
18416 315 pv-train-0000018416
18417 flipUD pv-train-0000018417
18418 45 pv-train-0000018418
18419 90 pv-train-0000018419
18420 135 pv-train-0000018420
18421 180 pv-train-0000018421
18422 225 pv-train-0000018422
18423 270 pv-train-0000018423
18424 315 pv-train-0000018424
18425 flipUD pv-train-0000018425
18426 45 pv-train-0000018426
18427 90 pv-train-0000018427
18428 135 pv-train-0000018428
18429 180 pv-train-0000018429
18430 225 pv-train-0000018430
18431 270 pv-train-0000018431
18432 315 pv-train-0000018432
images/1574565f-f45b-4160-947e-25ce1a81aaca.png
pv-test-0000000577
images/3dad6483-ac30-4d2b-b045-

18655 270 pv-train-0000018655
18656 315 pv-train-0000018656
images/3ff98212-7c2a-46cd-afb7-34a2479bed07.png
pv-test-0000000584
images/c7ec51d5-9f45-4209-8357-39e9a984a733.png
pv-train-0000018657
18658 45 pv-train-0000018658
18659 90 pv-train-0000018659
18660 135 pv-train-0000018660
18661 180 pv-train-0000018661
18662 225 pv-train-0000018662
18663 270 pv-train-0000018663
18664 315 pv-train-0000018664
18665 flipLR pv-train-0000018665
18666 45 pv-train-0000018666
18667 90 pv-train-0000018667
18668 135 pv-train-0000018668
18669 180 pv-train-0000018669
18670 225 pv-train-0000018670
18671 270 pv-train-0000018671
18672 315 pv-train-0000018672
18673 flipUD pv-train-0000018673
18674 45 pv-train-0000018674
18675 90 pv-train-0000018675
18676 135 pv-train-0000018676
18677 180 pv-train-0000018677
18678 225 pv-train-0000018678
18679 270 pv-train-0000018679
18680 315 pv-train-0000018680
18681 flipUD pv-train-0000018681
18682 45 pv-train-0000018682
18683 90 pv-train-0000018683
18684 135 pv-train-00000

18901 180 pv-train-0000018901
18902 225 pv-train-0000018902
18903 270 pv-train-0000018903
18904 315 pv-train-0000018904
18905 flipUD pv-train-0000018905
18906 45 pv-train-0000018906
18907 90 pv-train-0000018907
18908 135 pv-train-0000018908
18909 180 pv-train-0000018909
18910 225 pv-train-0000018910
18911 270 pv-train-0000018911
18912 315 pv-train-0000018912
images/72029aca-f0bc-477f-b790-f345f86228ce.png
pv-test-0000000592
images/c94827ce-d619-4705-a68a-df63a2c95ffa.png
pv-train-0000018913
18914 45 pv-train-0000018914
18915 90 pv-train-0000018915
18916 135 pv-train-0000018916
18917 180 pv-train-0000018917
18918 225 pv-train-0000018918
18919 270 pv-train-0000018919
18920 315 pv-train-0000018920
18921 flipLR pv-train-0000018921
18922 45 pv-train-0000018922
18923 90 pv-train-0000018923
18924 135 pv-train-0000018924
18925 180 pv-train-0000018925
18926 225 pv-train-0000018926
18927 270 pv-train-0000018927
18928 315 pv-train-0000018928
18929 flipUD pv-train-0000018929
18930 45 pv-train-0000

19147 90 pv-train-0000019147
19148 135 pv-train-0000019148
19149 180 pv-train-0000019149
19150 225 pv-train-0000019150
19151 270 pv-train-0000019151
19152 315 pv-train-0000019152
19153 flipUD pv-train-0000019153
19154 45 pv-train-0000019154
19155 90 pv-train-0000019155
19156 135 pv-train-0000019156
19157 180 pv-train-0000019157
19158 225 pv-train-0000019158
19159 270 pv-train-0000019159
19160 315 pv-train-0000019160
19161 flipUD pv-train-0000019161
19162 45 pv-train-0000019162
19163 90 pv-train-0000019163
19164 135 pv-train-0000019164
19165 180 pv-train-0000019165
19166 225 pv-train-0000019166
19167 270 pv-train-0000019167
19168 315 pv-train-0000019168
images/a2fc0093-8645-43c7-bdee-ec8ca9484815.png
pv-test-0000000600
images/161978c3-3a05-47a5-9fee-be7d25693eae.png
pv-train-0000019169
19170 45 pv-train-0000019170
19171 90 pv-train-0000019171
19172 135 pv-train-0000019172
19173 180 pv-train-0000019173
19174 225 pv-train-0000019174
19175 270 pv-train-0000019175
19176 315 pv-train-0000019

In [4]:
print(phase_count["train"])
print(phase_count["test"])

19329
605


In [5]:
# Training and Validate Sets
# degs = [45,90,135,180,225,270,315]
img_bbs = []
# phase_count["train"] = 3484
labels = os.path.join('test.json')
with open(labels,'r') as json_file:
    datas = json.load(json_file)
    for data in datas:
        read_path = (data['image']['pathname'])[1:]
        print(read_path)
        phase = "train"
        img_name_new = "pv-" + phase + "-{:010d}".format(phase_count[phase])
        print(img_name_new)
        save_path = os.path.join('../data/custom/images',img_name_new+".jpg")
        img = Image.open(read_path)
        objects = data['objects']
        rects = []
        result = ""
        img = np.array(img)
        h,w,c = img.shape
        for obj in objects:
            minr = obj['bounding_box']['minimum']['r']
            minc = obj['bounding_box']['minimum']['c']
            maxr = obj['bounding_box']['maximum']['r']
            maxc = obj['bounding_box']['maximum']['c']
            x1 = minc / w
            y1 = minr / h
            bw = (maxc - minc) / w
            bh = (maxr - minr) / h
            xc = x1 + (bw / 2)
            yc = y1 + (bh / 2)
            category = obj['category']
            if (category == "red blood cell"):
                result += "\n"+ "0 " + str(xc) + " " + str(yc) + " " + str(bw) + " " + str(bh)
                rects.append(BoundingBox(x1=minc,y1=minr,x2=maxc,y2=maxr,label=0))
            elif category == "ring" or category == "trophozoite" or category == "schizont" or category == "gametocyte":
                result += "\n" + "1 " + str(xc) + " " + str(yc) + " " + str(bw) + " " + str(bh)
                rects.append(BoundingBox(x1=minc,y1=minr,x2=maxc,y2=maxr,label=1))
        result = result[1:]
        Image.fromarray(img).save(save_path)
        f = open(os.path.join('../data/custom/labels',img_name_new+".txt"),'w')
        f.write(result)
        f.close()
        bbs = BoundingBoxesOnImage(rects,shape=img.shape)
        count += 1
        phase_count[phase] += 1
        
        phase_count = rotate(img=img,bbs=bbs,specie="pv",offset=10,phase_count=phase_count)
        
        aug_hor, bbs_hor, phase_count = flipHor(img=img,bbs=bbs,specie="pv",phase_count=phase_count)
        phase_count = rotate(img=aug_hor,bbs=bbs_hor,specie="pv",offset=10,phase_count=phase_count)
        
        aug_ver, bbs_ver, phase_count = flipVer(img=aug_hor,bbs=bbs_hor,specie="pv",phase_count=phase_count)
        phase_count = rotate(img=aug_ver,bbs=bbs_ver,specie="pv",offset=10,phase_count=phase_count)
        
        aug_ver2, bbs_ver2, phase_count = flipVer(img=img,bbs=bbs,specie="pv",phase_count=phase_count)
        phase_count = rotate(img=aug_ver2,bbs=bbs_ver2,specie="pv",offset=10,phase_count=phase_count)

images/41be1bd3-0d31-4881-bf1f-3ccdfa21ff12.jpg
pv-train-0000019329
19330 45 pv-train-0000019330
19331 90 pv-train-0000019331
19332 135 pv-train-0000019332
19333 180 pv-train-0000019333
19334 225 pv-train-0000019334
19335 270 pv-train-0000019335
19336 315 pv-train-0000019336
19337 flipLR pv-train-0000019337
19338 45 pv-train-0000019338
19339 90 pv-train-0000019339
19340 135 pv-train-0000019340
19341 180 pv-train-0000019341
19342 225 pv-train-0000019342
19343 270 pv-train-0000019343
19344 315 pv-train-0000019344
19345 flipUD pv-train-0000019345
19346 45 pv-train-0000019346
19347 90 pv-train-0000019347
19348 135 pv-train-0000019348
19349 180 pv-train-0000019349
19350 225 pv-train-0000019350
19351 270 pv-train-0000019351
19352 315 pv-train-0000019352
19353 flipUD pv-train-0000019353
19354 45 pv-train-0000019354
19355 90 pv-train-0000019355
19356 135 pv-train-0000019356
19357 180 pv-train-0000019357
19358 225 pv-train-0000019358
19359 270 pv-train-0000019359
19360 315 pv-train-0000019360
i

19594 45 pv-train-0000019594
19595 90 pv-train-0000019595
19596 135 pv-train-0000019596
19597 180 pv-train-0000019597
19598 225 pv-train-0000019598
19599 270 pv-train-0000019599
19600 315 pv-train-0000019600
19601 flipUD pv-train-0000019601
19602 45 pv-train-0000019602
19603 90 pv-train-0000019603
19604 135 pv-train-0000019604
19605 180 pv-train-0000019605
19606 225 pv-train-0000019606
19607 270 pv-train-0000019607
19608 315 pv-train-0000019608
19609 flipUD pv-train-0000019609
19610 45 pv-train-0000019610
19611 90 pv-train-0000019611
19612 135 pv-train-0000019612
19613 180 pv-train-0000019613
19614 225 pv-train-0000019614
19615 270 pv-train-0000019615
19616 315 pv-train-0000019616
images/8298aca0-5099-4d45-88ca-ec8ac7922e77.jpg
pv-train-0000019617
19618 45 pv-train-0000019618
19619 90 pv-train-0000019619
19620 135 pv-train-0000019620
19621 180 pv-train-0000019621
19622 225 pv-train-0000019622
19623 270 pv-train-0000019623
19624 315 pv-train-0000019624
19625 flipLR pv-train-0000019625
1

19858 45 pv-train-0000019858
19859 90 pv-train-0000019859
19860 135 pv-train-0000019860
19861 180 pv-train-0000019861
19862 225 pv-train-0000019862
19863 270 pv-train-0000019863
19864 315 pv-train-0000019864
19865 flipUD pv-train-0000019865
19866 45 pv-train-0000019866
19867 90 pv-train-0000019867
19868 135 pv-train-0000019868
19869 180 pv-train-0000019869
19870 225 pv-train-0000019870
19871 270 pv-train-0000019871
19872 315 pv-train-0000019872
images/54c88d45-30c3-46f8-8fef-b3e9c07781f2.jpg
pv-train-0000019873
19874 45 pv-train-0000019874
19875 90 pv-train-0000019875
19876 135 pv-train-0000019876
19877 180 pv-train-0000019877
19878 225 pv-train-0000019878
19879 270 pv-train-0000019879
19880 315 pv-train-0000019880
19881 flipLR pv-train-0000019881
19882 45 pv-train-0000019882
19883 90 pv-train-0000019883
19884 135 pv-train-0000019884
19885 180 pv-train-0000019885
19886 225 pv-train-0000019886
19887 270 pv-train-0000019887
19888 315 pv-train-0000019888
19889 flipUD pv-train-0000019889
1

20122 45 pv-train-0000020122
20123 90 pv-train-0000020123
20124 135 pv-train-0000020124
20125 180 pv-train-0000020125
20126 225 pv-train-0000020126
20127 270 pv-train-0000020127
20128 315 pv-train-0000020128
images/511b6e13-acd7-4dd3-b1d0-c4aa10a514c7.jpg
pv-train-0000020129
20130 45 pv-train-0000020130
20131 90 pv-train-0000020131
20132 135 pv-train-0000020132
20133 180 pv-train-0000020133
20134 225 pv-train-0000020134
20135 270 pv-train-0000020135
20136 315 pv-train-0000020136
20137 flipLR pv-train-0000020137
20138 45 pv-train-0000020138
20139 90 pv-train-0000020139
20140 135 pv-train-0000020140
20141 180 pv-train-0000020141
20142 225 pv-train-0000020142
20143 270 pv-train-0000020143
20144 315 pv-train-0000020144
20145 flipUD pv-train-0000020145
20146 45 pv-train-0000020146
20147 90 pv-train-0000020147
20148 135 pv-train-0000020148
20149 180 pv-train-0000020149
20150 225 pv-train-0000020150
20151 270 pv-train-0000020151
20152 315 pv-train-0000020152
20153 flipUD pv-train-0000020153
2

20386 45 pv-train-0000020386
20387 90 pv-train-0000020387
20388 135 pv-train-0000020388
20389 180 pv-train-0000020389
20390 225 pv-train-0000020390
20391 270 pv-train-0000020391
20392 315 pv-train-0000020392
20393 flipLR pv-train-0000020393
20394 45 pv-train-0000020394
20395 90 pv-train-0000020395
20396 135 pv-train-0000020396
20397 180 pv-train-0000020397
20398 225 pv-train-0000020398
20399 270 pv-train-0000020399
20400 315 pv-train-0000020400
20401 flipUD pv-train-0000020401
20402 45 pv-train-0000020402
20403 90 pv-train-0000020403
20404 135 pv-train-0000020404
20405 180 pv-train-0000020405
20406 225 pv-train-0000020406
20407 270 pv-train-0000020407
20408 315 pv-train-0000020408
20409 flipUD pv-train-0000020409
20410 45 pv-train-0000020410
20411 90 pv-train-0000020411
20412 135 pv-train-0000020412
20413 180 pv-train-0000020413
20414 225 pv-train-0000020414
20415 270 pv-train-0000020415
20416 315 pv-train-0000020416
images/e02301ac-68f7-4daf-b2ba-254b917f1cca.jpg
pv-train-0000020417
2

...

21971 90 pv-train-0000021971
21972 135 pv-train-0000021972
21973 180 pv-train-0000021973
21974 225 pv-train-0000021974
21975 270 pv-train-0000021975
21976 315 pv-train-0000021976
21977 flipUD pv-train-0000021977
21978 45 pv-train-0000021978
21979 90 pv-train-0000021979
21980 135 pv-train-0000021980
21981 180 pv-train-0000021981
21982 225 pv-train-0000021982
21983 270 pv-train-0000021983
21984 315 pv-train-0000021984
images/2c03b3c9-7786-4e1b-ac2b-c98e9f8358f2.jpg
pv-train-0000021985
21986 45 pv-train-0000021986
21987 90 pv-train-0000021987
21988 135 pv-train-0000021988
21989 180 pv-train-0000021989
21990 225 pv-train-0000021990
21991 270 pv-train-0000021991
21992 315 pv-train-0000021992
21993 flipLR pv-train-0000021993
21994 45 pv-train-0000021994
21995 90 pv-train-0000021995
21996 135 pv-train-0000021996
21997 180 pv-train-0000021997
21998 225 pv-train-0000021998
21999 270 pv-train-0000021999
22000 315 pv-train-0000022000
22001 flipUD pv-train-0000022001
22002 45 pv-train-0000022002
2

22236 135 pv-train-0000022236
22237 180 pv-train-0000022237
22238 225 pv-train-0000022238
22239 270 pv-train-0000022239
22240 315 pv-train-0000022240
images/5bfde411-aa51-45d0-8b67-a57c5f1e628b.jpg
pv-train-0000022241
22242 45 pv-train-0000022242
22243 90 pv-train-0000022243
22244 135 pv-train-0000022244
22245 180 pv-train-0000022245
22246 225 pv-train-0000022246
22247 270 pv-train-0000022247
22248 315 pv-train-0000022248
22249 flipLR pv-train-0000022249
22250 45 pv-train-0000022250
22251 90 pv-train-0000022251
22252 135 pv-train-0000022252
22253 180 pv-train-0000022253
22254 225 pv-train-0000022254
22255 270 pv-train-0000022255
22256 315 pv-train-0000022256
22257 flipUD pv-train-0000022257
22258 45 pv-train-0000022258
22259 90 pv-train-0000022259
22260 135 pv-train-0000022260
22261 180 pv-train-0000022261
22262 225 pv-train-0000022262
22263 270 pv-train-0000022263
22264 315 pv-train-0000022264
22265 flipUD pv-train-0000022265
22266 45 pv-train-0000022266
22267 90 pv-train-0000022267
2

22498 45 pv-train-0000022498
22499 90 pv-train-0000022499
22500 135 pv-train-0000022500
22501 180 pv-train-0000022501
22502 225 pv-train-0000022502
22503 270 pv-train-0000022503
22504 315 pv-train-0000022504
22505 flipLR pv-train-0000022505
22506 45 pv-train-0000022506
22507 90 pv-train-0000022507
22508 135 pv-train-0000022508
22509 180 pv-train-0000022509
22510 225 pv-train-0000022510
22511 270 pv-train-0000022511
22512 315 pv-train-0000022512
22513 flipUD pv-train-0000022513
22514 45 pv-train-0000022514
22515 90 pv-train-0000022515
22516 135 pv-train-0000022516
22517 180 pv-train-0000022517
22518 225 pv-train-0000022518
22519 270 pv-train-0000022519
22520 315 pv-train-0000022520
22521 flipUD pv-train-0000022521
22522 45 pv-train-0000022522
22523 90 pv-train-0000022523
22524 135 pv-train-0000022524
22525 180 pv-train-0000022525
22526 225 pv-train-0000022526
22527 270 pv-train-0000022527
22528 315 pv-train-0000022528
images/9cf3f75a-354c-46f9-8b53-0a9711ac8bee.jpg
pv-train-0000022529
2

22763 90 pv-train-0000022763
22764 135 pv-train-0000022764
22765 180 pv-train-0000022765
22766 225 pv-train-0000022766
22767 270 pv-train-0000022767
22768 315 pv-train-0000022768
22769 flipUD pv-train-0000022769
22770 45 pv-train-0000022770
22771 90 pv-train-0000022771
22772 135 pv-train-0000022772
22773 180 pv-train-0000022773
22774 225 pv-train-0000022774
22775 270 pv-train-0000022775
22776 315 pv-train-0000022776
22777 flipUD pv-train-0000022777
22778 45 pv-train-0000022778
22779 90 pv-train-0000022779
22780 135 pv-train-0000022780
22781 180 pv-train-0000022781
22782 225 pv-train-0000022782
22783 270 pv-train-0000022783
22784 315 pv-train-0000022784
images/4da8f6c5-5f85-4280-92d0-1ba008e8c404.jpg
pv-train-0000022785
22786 45 pv-train-0000022786
22787 90 pv-train-0000022787
22788 135 pv-train-0000022788
22789 180 pv-train-0000022789
22790 225 pv-train-0000022790
22791 270 pv-train-0000022791
22792 315 pv-train-0000022792
22793 flipLR pv-train-0000022793
22794 45 pv-train-0000022794
2

23028 135 pv-train-0000023028
23029 180 pv-train-0000023029
23030 225 pv-train-0000023030
23031 270 pv-train-0000023031
23032 315 pv-train-0000023032
23033 flipUD pv-train-0000023033
23034 45 pv-train-0000023034
23035 90 pv-train-0000023035
23036 135 pv-train-0000023036
23037 180 pv-train-0000023037
23038 225 pv-train-0000023038
23039 270 pv-train-0000023039
23040 315 pv-train-0000023040
images/1b814efc-7bf9-40ae-8e13-49dc2c04732f.jpg
pv-train-0000023041
23042 45 pv-train-0000023042
23043 90 pv-train-0000023043
23044 135 pv-train-0000023044
23045 180 pv-train-0000023045
23046 225 pv-train-0000023046
23047 270 pv-train-0000023047
23048 315 pv-train-0000023048
23049 flipLR pv-train-0000023049
23050 45 pv-train-0000023050
23051 90 pv-train-0000023051
23052 135 pv-train-0000023052
23053 180 pv-train-0000023053
23054 225 pv-train-0000023054
23055 270 pv-train-0000023055
23056 315 pv-train-0000023056
23057 flipUD pv-train-0000023057
23058 45 pv-train-0000023058
23059 90 pv-train-0000023059
2

In [6]:
print(phase_count["train"])
print(phase_count["test"])

23169
605


In [7]:
print(phase_count["train"])
print(phase_count["test"])

23169
605


In [8]:
# labels = os.path.join('pv_ext.json')
phase = "train"

data = (requests.get('https://raw.githubusercontent.com/RTANC/malaria_dataset/master/Vivax/pv_ext.json')).json()
images = data["_via_img_metadata"]
for key in images:
    url = images[key]["filename"]
    leng = len(images[key]["regions"])
    bboxs = ""
    if leng > 0:
        name = "pv-" + phase + "-{:010d}".format(phase_count[phase])
        print("{} {}".format(phase_count[phase],url))
        print(name)
        response = requests.get(url)
        img = Image.open(BytesIO(response.content))
        w, h = img.size
        img.save(os.path.join('../data/custom/images',name+".jpg"))
        f = open(os.path.join('../data/custom/labels',name+".txt"),'w')
        rects = []
        for box in images[key]["regions"]:
            b = box["shape_attributes"]
            x1 = b["x"]/w
            y1 = b["y"]/h
            bw = b["width"]/w
            bh = b["height"]/h
            xc = x1 + (bw/2)
            yc = y1 + (bh/2)
            lbl = box["region_attributes"]["type"]
            bboxs += "\n"+ lbl + " " + str(xc) + " " + str(yc) + " " + str(bw) + " " + str(bh)
            rects.append(BoundingBox(x1=b["x"],y1=b["y"],x2=(b["x"] + b["width"]),y2=(b["y"] + b["height"]),label=lbl))
        bboxs = bboxs[1:]
        f.write(bboxs)
        f.close()
        img = np.array(img)
        bbs = BoundingBoxesOnImage(rects,shape=img.shape)
        phase_count[phase] += 1
        
        phase_count = rotate(img=img,bbs=bbs,specie="pv",offset=20,phase_count=phase_count)
        
        aug_hor, bbs_hor, phase_count = flipHor(img=img,bbs=bbs,specie="pv",phase_count=phase_count)
        phase_count = rotate(img=aug_hor,bbs=bbs_hor,specie="pv",offset=20,phase_count=phase_count)
        
        aug_ver, bbs_ver, phase_count = flipVer(img=aug_hor,bbs=bbs_hor,specie="pv",phase_count=phase_count)
        phase_count = rotate(img=aug_ver,bbs=bbs_ver,specie="pv",offset=20,phase_count=phase_count)
        
        aug_ver2, bbs_ver2, phase_count = flipVer(img=img,bbs=bbs,specie="pv",phase_count=phase_count)
        phase_count = rotate(img=aug_ver2,bbs=bbs_ver2,specie="pv",offset=20,phase_count=phase_count)

23169 https://raw.githubusercontent.com/RTANC/malaria_dataset/master/Vivax/pv-0000000001.jpg
pv-train-0000023169
23170 45 pv-train-0000023170
23171 90 pv-train-0000023171
23172 135 pv-train-0000023172
23173 180 pv-train-0000023173
23174 225 pv-train-0000023174
23175 270 pv-train-0000023175
23176 315 pv-train-0000023176
23177 flipLR pv-train-0000023177
23178 45 pv-train-0000023178
23179 90 pv-train-0000023179
23180 135 pv-train-0000023180
23181 180 pv-train-0000023181
23182 225 pv-train-0000023182
23183 270 pv-train-0000023183
23184 315 pv-train-0000023184
23185 flipUD pv-train-0000023185
23186 45 pv-train-0000023186
23187 90 pv-train-0000023187
23188 135 pv-train-0000023188
23189 180 pv-train-0000023189
23190 225 pv-train-0000023190
23191 270 pv-train-0000023191
23192 315 pv-train-0000023192
23193 flipUD pv-train-0000023193
23194 45 pv-train-0000023194
23195 90 pv-train-0000023195
23196 135 pv-train-0000023196
23197 180 pv-train-0000023197
23198 225 pv-train-0000023198
23199 270 pv-tra

23420 135 pv-train-0000023420
23421 180 pv-train-0000023421
23422 225 pv-train-0000023422
23423 270 pv-train-0000023423
23424 315 pv-train-0000023424
23425 https://raw.githubusercontent.com/RTANC/malaria_dataset/master/Vivax/pv-0000000010.jpg
pv-train-0000023425
23426 45 pv-train-0000023426
23427 90 pv-train-0000023427
23428 135 pv-train-0000023428
23429 180 pv-train-0000023429
23430 225 pv-train-0000023430
23431 270 pv-train-0000023431
23432 315 pv-train-0000023432
23433 flipLR pv-train-0000023433
23434 45 pv-train-0000023434
23435 90 pv-train-0000023435
23436 135 pv-train-0000023436
23437 180 pv-train-0000023437
23438 225 pv-train-0000023438
23439 270 pv-train-0000023439
23440 315 pv-train-0000023440
23441 flipUD pv-train-0000023441
23442 45 pv-train-0000023442
23443 90 pv-train-0000023443
23444 135 pv-train-0000023444
23445 180 pv-train-0000023445
23446 225 pv-train-0000023446
23447 270 pv-train-0000023447
23448 315 pv-train-0000023448
23449 flipUD pv-train-0000023449
23450 45 pv-tr

23673 flipUD pv-train-0000023673
23674 45 pv-train-0000023674
23675 90 pv-train-0000023675
23676 135 pv-train-0000023676
23677 180 pv-train-0000023677
23678 225 pv-train-0000023678
23679 270 pv-train-0000023679
23680 315 pv-train-0000023680
23681 https://raw.githubusercontent.com/RTANC/malaria_dataset/master/Vivax/pv-0000000018.jpg
pv-train-0000023681
23682 45 pv-train-0000023682
23683 90 pv-train-0000023683
23684 135 pv-train-0000023684
23685 180 pv-train-0000023685
23686 225 pv-train-0000023686
23687 270 pv-train-0000023687
23688 315 pv-train-0000023688
23689 flipLR pv-train-0000023689
23690 45 pv-train-0000023690
23691 90 pv-train-0000023691
23692 135 pv-train-0000023692
23693 180 pv-train-0000023693
23694 225 pv-train-0000023694
23695 270 pv-train-0000023695
23696 315 pv-train-0000023696
23697 flipUD pv-train-0000023697
23698 45 pv-train-0000023698
23699 90 pv-train-0000023699
23700 135 pv-train-0000023700
23701 180 pv-train-0000023701
23702 225 pv-train-0000023702
23703 270 pv-tra

23927 270 pv-train-0000023927
23928 315 pv-train-0000023928
23929 flipUD pv-train-0000023929
23930 45 pv-train-0000023930
23931 90 pv-train-0000023931
23932 135 pv-train-0000023932
23933 180 pv-train-0000023933
23934 225 pv-train-0000023934
23935 270 pv-train-0000023935
23936 315 pv-train-0000023936
23937 https://raw.githubusercontent.com/RTANC/malaria_dataset/master/Vivax/pv-0000000026.jpg
pv-train-0000023937
23938 45 pv-train-0000023938
23939 90 pv-train-0000023939
23940 135 pv-train-0000023940
23941 180 pv-train-0000023941
23942 225 pv-train-0000023942
23943 270 pv-train-0000023943
23944 315 pv-train-0000023944
23945 flipLR pv-train-0000023945
23946 45 pv-train-0000023946
23947 90 pv-train-0000023947
23948 135 pv-train-0000023948
23949 180 pv-train-0000023949
23950 225 pv-train-0000023950
23951 270 pv-train-0000023951
23952 315 pv-train-0000023952
23953 flipUD pv-train-0000023953
23954 45 pv-train-0000023954
23955 90 pv-train-0000023955
23956 135 pv-train-0000023956
23957 180 pv-tra

24182 225 pv-train-0000024182
24183 270 pv-train-0000024183
24184 315 pv-train-0000024184
24185 flipUD pv-train-0000024185
24186 45 pv-train-0000024186
24187 90 pv-train-0000024187
24188 135 pv-train-0000024188
24189 180 pv-train-0000024189
24190 225 pv-train-0000024190
24191 270 pv-train-0000024191
24192 315 pv-train-0000024192
24193 https://raw.githubusercontent.com/RTANC/malaria_dataset/master/Vivax/pv-0000000034.jpg
pv-train-0000024193
24194 45 pv-train-0000024194
24195 90 pv-train-0000024195
24196 135 pv-train-0000024196
24197 180 pv-train-0000024197
24198 225 pv-train-0000024198
24199 270 pv-train-0000024199
24200 315 pv-train-0000024200
24201 flipLR pv-train-0000024201
24202 45 pv-train-0000024202
24203 90 pv-train-0000024203
24204 135 pv-train-0000024204
24205 180 pv-train-0000024205
24206 225 pv-train-0000024206
24207 270 pv-train-0000024207
24208 315 pv-train-0000024208
24209 flipUD pv-train-0000024209
24210 45 pv-train-0000024210
24211 90 pv-train-0000024211
24212 135 pv-tra

...

44052 45 pv-train-0000044052
44053 90 pv-train-0000044053
44054 135 pv-train-0000044054
44055 180 pv-train-0000044055
44056 225 pv-train-0000044056
44057 270 pv-train-0000044057
44058 315 pv-train-0000044058
44059 flipUD pv-train-0000044059
44060 45 pv-train-0000044060
44061 90 pv-train-0000044061
44062 135 pv-train-0000044062
44063 180 pv-train-0000044063
44064 225 pv-train-0000044064
44065 270 pv-train-0000044065
44066 315 pv-train-0000044066
44067 flipUD pv-train-0000044067
44068 45 pv-train-0000044068
44069 90 pv-train-0000044069
44070 135 pv-train-0000044070
44071 180 pv-train-0000044071
44072 225 pv-train-0000044072
44073 270 pv-train-0000044073
44074 315 pv-train-0000044074
44075 https://raw.githubusercontent.com/RTANC/malaria_dataset/master/Vivax/pv-0000000786.jpg
pv-train-0000044075
44076 45 pv-train-0000044076
44077 90 pv-train-0000044077
44078 135 pv-train-0000044078
44079 180 pv-train-0000044079
44080 225 pv-train-0000044080
44081 270 pv-train-0000044081
44082 315 pv-train-

44305 270 pv-train-0000044305
44306 315 pv-train-0000044306
44307 flipUD pv-train-0000044307
44308 45 pv-train-0000044308
44309 90 pv-train-0000044309
44310 135 pv-train-0000044310
44311 180 pv-train-0000044311
44312 225 pv-train-0000044312
44313 270 pv-train-0000044313
44314 315 pv-train-0000044314
44315 https://raw.githubusercontent.com/RTANC/malaria_dataset/master/Vivax/pv-0000000794.jpg
pv-train-0000044315
44316 45 pv-train-0000044316
44317 90 pv-train-0000044317
44318 135 pv-train-0000044318
44319 180 pv-train-0000044319
44320 225 pv-train-0000044320
44321 270 pv-train-0000044321
44322 315 pv-train-0000044322
44323 flipLR pv-train-0000044323
44324 45 pv-train-0000044324
44325 90 pv-train-0000044325
44326 135 pv-train-0000044326
44327 180 pv-train-0000044327
44328 225 pv-train-0000044328
44329 270 pv-train-0000044329
44330 315 pv-train-0000044330
44331 flipUD pv-train-0000044331
44332 45 pv-train-0000044332
44333 90 pv-train-0000044333
44334 135 pv-train-0000044334
44335 180 pv-tra

44551 270 pv-train-0000044551
44552 315 pv-train-0000044552
44553 flipUD pv-train-0000044553
44554 45 pv-train-0000044554
44555 90 pv-train-0000044555
44556 180 pv-train-0000044556
44557 225 pv-train-0000044557
44558 270 pv-train-0000044558
44559 flipUD pv-train-0000044559
44560 90 pv-train-0000044560
44561 135 pv-train-0000044561
44562 180 pv-train-0000044562
44563 270 pv-train-0000044563
44564 315 pv-train-0000044564
44565 https://raw.githubusercontent.com/RTANC/malaria_dataset/master/Vivax/pv-0000000804.jpg
pv-train-0000044565
44566 180 pv-train-0000044566
44567 flipLR pv-train-0000044567
44568 180 pv-train-0000044568
44569 flipUD pv-train-0000044569
44570 180 pv-train-0000044570
44571 flipUD pv-train-0000044571
44572 180 pv-train-0000044572
44573 https://raw.githubusercontent.com/RTANC/malaria_dataset/master/Vivax/pv-0000000805.jpg
pv-train-0000044573
44574 90 pv-train-0000044574
44575 135 pv-train-0000044575
44576 180 pv-train-0000044576
44577 270 pv-train-0000044577
44578 315 pv-

44798 180 pv-train-0000044798
44799 flipLR pv-train-0000044799
44800 180 pv-train-0000044800
44801 flipUD pv-train-0000044801
44802 180 pv-train-0000044802
44803 flipUD pv-train-0000044803
44804 180 pv-train-0000044804
44805 https://raw.githubusercontent.com/RTANC/malaria_dataset/master/Vivax/pv-0000000814.jpg
pv-train-0000044805
44806 45 pv-train-0000044806
44807 90 pv-train-0000044807
44808 135 pv-train-0000044808
44809 180 pv-train-0000044809
44810 225 pv-train-0000044810
44811 270 pv-train-0000044811
44812 315 pv-train-0000044812
44813 flipLR pv-train-0000044813
44814 45 pv-train-0000044814
44815 90 pv-train-0000044815
44816 135 pv-train-0000044816
44817 180 pv-train-0000044817
44818 225 pv-train-0000044818
44819 270 pv-train-0000044819
44820 315 pv-train-0000044820
44821 flipUD pv-train-0000044821
44822 45 pv-train-0000044822
44823 90 pv-train-0000044823
44824 135 pv-train-0000044824
44825 180 pv-train-0000044825
44826 225 pv-train-0000044826
44827 270 pv-train-0000044827
44828 31

45051 270 pv-train-0000045051
45052 315 pv-train-0000045052
45053 flipUD pv-train-0000045053
45054 45 pv-train-0000045054
45055 90 pv-train-0000045055
45056 135 pv-train-0000045056
45057 180 pv-train-0000045057
45058 225 pv-train-0000045058
45059 270 pv-train-0000045059
45060 315 pv-train-0000045060
45061 https://raw.githubusercontent.com/RTANC/malaria_dataset/master/Vivax/pv-0000000822.jpg
pv-train-0000045061
45062 45 pv-train-0000045062
45063 90 pv-train-0000045063
45064 135 pv-train-0000045064
45065 180 pv-train-0000045065
45066 225 pv-train-0000045066
45067 270 pv-train-0000045067
45068 315 pv-train-0000045068
45069 flipLR pv-train-0000045069
45070 45 pv-train-0000045070
45071 90 pv-train-0000045071
45072 135 pv-train-0000045072
45073 180 pv-train-0000045073
45074 225 pv-train-0000045074
45075 270 pv-train-0000045075
45076 315 pv-train-0000045076
45077 flipUD pv-train-0000045077
45078 45 pv-train-0000045078
45079 90 pv-train-0000045079
45080 135 pv-train-0000045080
45081 180 pv-tra

In [9]:
phase = "train"

data = (requests.get('https://raw.githubusercontent.com/RTANC/malaria_dataset/master/Vivax/pv-09.json')).json()
images = data["_via_img_metadata"]
for key in images:
    url = images[key]["filename"]
    leng = len(images[key]["regions"])
    bboxs = ""
    if leng > 0:
        name = "pv-" + phase + "-{:010d}".format(phase_count[phase])
        print("{} {}".format(phase_count[phase],url))
        print(name)
        response = requests.get(url)
        img = Image.open(BytesIO(response.content))
        w, h = img.size
        img.save(os.path.join('../data/custom/images',name+".jpg"))
        f = open(os.path.join('../data/custom/labels',name+".txt"),'w')
        rects = []
        for box in images[key]["regions"]:
            b = box["shape_attributes"]
            x1 = b["x"]/w
            y1 = b["y"]/h
            bw = b["width"]/w
            bh = b["height"]/h
            xc = x1 + (bw/2)
            yc = y1 + (bh/2)
            lbl = box["region_attributes"]["type"]
            bboxs += "\n"+ lbl + " " + str(xc) + " " + str(yc) + " " + str(bw) + " " + str(bh)
            rects.append(BoundingBox(x1=b["x"],y1=b["y"],x2=(b["x"] + b["width"]),y2=(b["y"] + b["height"]),label=lbl))
        bboxs = bboxs[1:]
        f.write(bboxs)
        f.close()
        img = np.array(img)
        bbs = BoundingBoxesOnImage(rects,shape=img.shape)
        phase_count[phase] += 1
        
        phase_count = rotate(img=img,bbs=bbs,specie="pv",offset=20,phase_count=phase_count)
        
        aug_hor, bbs_hor, phase_count = flipHor(img=img,bbs=bbs,specie="pv",phase_count=phase_count)
        phase_count = rotate(img=aug_hor,bbs=bbs_hor,specie="pv",offset=20,phase_count=phase_count)
        
        aug_ver, bbs_ver, phase_count = flipVer(img=aug_hor,bbs=bbs_hor,specie="pv",phase_count=phase_count)
        phase_count = rotate(img=aug_ver,bbs=bbs_ver,specie="pv",offset=20,phase_count=phase_count)
        
        aug_ver2, bbs_ver2, phase_count = flipVer(img=img,bbs=bbs,specie="pv",phase_count=phase_count)
        phase_count = rotate(img=aug_ver2,bbs=bbs_ver2,specie="pv",offset=20,phase_count=phase_count)

45149 https://raw.githubusercontent.com/RTANC/malaria_dataset/master/Vivax/pv-09/Data20200117-09_Wholeslide_Default_Extended_i09j65.jpg
pv-train-0000045149
45150 45 pv-train-0000045150
45151 90 pv-train-0000045151
45152 135 pv-train-0000045152
45153 180 pv-train-0000045153
45154 225 pv-train-0000045154
45155 270 pv-train-0000045155
45156 315 pv-train-0000045156
45157 flipLR pv-train-0000045157
45158 45 pv-train-0000045158
45159 90 pv-train-0000045159
45160 135 pv-train-0000045160
45161 180 pv-train-0000045161
45162 225 pv-train-0000045162
45163 270 pv-train-0000045163
45164 315 pv-train-0000045164
45165 flipUD pv-train-0000045165
45166 45 pv-train-0000045166
45167 90 pv-train-0000045167
45168 135 pv-train-0000045168
45169 180 pv-train-0000045169
45170 225 pv-train-0000045170
45171 270 pv-train-0000045171
45172 315 pv-train-0000045172
45173 flipUD pv-train-0000045173
45174 45 pv-train-0000045174
45175 90 pv-train-0000045175
45176 135 pv-train-0000045176
45177 180 pv-train-0000045177
451

45385 225 pv-train-0000045385
45386 270 pv-train-0000045386
45387 flipLR pv-train-0000045387
45388 90 pv-train-0000045388
45389 135 pv-train-0000045389
45390 180 pv-train-0000045390
45391 270 pv-train-0000045391
45392 315 pv-train-0000045392
45393 flipUD pv-train-0000045393
45394 45 pv-train-0000045394
45395 90 pv-train-0000045395
45396 180 pv-train-0000045396
45397 225 pv-train-0000045397
45398 270 pv-train-0000045398
45399 flipUD pv-train-0000045399
45400 90 pv-train-0000045400
45401 135 pv-train-0000045401
45402 180 pv-train-0000045402
45403 270 pv-train-0000045403
45404 315 pv-train-0000045404
45405 https://raw.githubusercontent.com/RTANC/malaria_dataset/master/Vivax/pv-09/Data20200117-09_Wholeslide_Default_Extended_i13j59.jpg
pv-train-0000045405
45406 45 pv-train-0000045406
45407 90 pv-train-0000045407
45408 135 pv-train-0000045408
45409 180 pv-train-0000045409
45410 225 pv-train-0000045410
45411 270 pv-train-0000045411
45412 315 pv-train-0000045412
45413 flipLR pv-train-000004541

45624 135 pv-train-0000045624
45625 180 pv-train-0000045625
45626 225 pv-train-0000045626
45627 270 pv-train-0000045627
45628 315 pv-train-0000045628
45629 https://raw.githubusercontent.com/RTANC/malaria_dataset/master/Vivax/pv-09/Data20200117-09_Wholeslide_Default_Extended_i15j53.jpg
pv-train-0000045629
45630 90 pv-train-0000045630
45631 135 pv-train-0000045631
45632 180 pv-train-0000045632
45633 270 pv-train-0000045633
45634 315 pv-train-0000045634
45635 flipUD pv-train-0000045635
45636 45 pv-train-0000045636
45637 90 pv-train-0000045637
45638 225 pv-train-0000045638
45639 270 pv-train-0000045639
45640 https://raw.githubusercontent.com/RTANC/malaria_dataset/master/Vivax/pv-09/Data20200117-09_Wholeslide_Default_Extended_i15j61.jpg
pv-train-0000045640
45641 45 pv-train-0000045641
45642 90 pv-train-0000045642
45643 135 pv-train-0000045643
45644 180 pv-train-0000045644
45645 225 pv-train-0000045645
45646 270 pv-train-0000045646
45647 315 pv-train-0000045647
45648 flipLR pv-train-00000456

45855 315 pv-train-0000045855
45856 flipLR pv-train-0000045856
45857 45 pv-train-0000045857
45858 90 pv-train-0000045858
45859 135 pv-train-0000045859
45860 180 pv-train-0000045860
45861 225 pv-train-0000045861
45862 270 pv-train-0000045862
45863 315 pv-train-0000045863
45864 flipUD pv-train-0000045864
45865 45 pv-train-0000045865
45866 90 pv-train-0000045866
45867 135 pv-train-0000045867
45868 180 pv-train-0000045868
45869 225 pv-train-0000045869
45870 270 pv-train-0000045870
45871 315 pv-train-0000045871
45872 flipUD pv-train-0000045872
45873 45 pv-train-0000045873
45874 90 pv-train-0000045874
45875 135 pv-train-0000045875
45876 180 pv-train-0000045876
45877 225 pv-train-0000045877
45878 270 pv-train-0000045878
45879 315 pv-train-0000045879
45880 https://raw.githubusercontent.com/RTANC/malaria_dataset/master/Vivax/pv-09/Data20200117-09_Wholeslide_Default_Extended_i17j66.jpg
pv-train-0000045880
45881 45 pv-train-0000045881
45882 90 pv-train-0000045882
45883 135 pv-train-0000045883
458

46094 270 pv-train-0000046094
46095 315 pv-train-0000046095
46096 https://raw.githubusercontent.com/RTANC/malaria_dataset/master/Vivax/pv-09/Data20200117-09_Wholeslide_Default_Extended_i19j57.jpg
pv-train-0000046096
46097 45 pv-train-0000046097
46098 90 pv-train-0000046098
46099 135 pv-train-0000046099
46100 180 pv-train-0000046100
46101 225 pv-train-0000046101
46102 270 pv-train-0000046102
46103 315 pv-train-0000046103
46104 flipLR pv-train-0000046104
46105 45 pv-train-0000046105
46106 90 pv-train-0000046106
46107 135 pv-train-0000046107
46108 180 pv-train-0000046108
46109 225 pv-train-0000046109
46110 270 pv-train-0000046110
46111 315 pv-train-0000046111
46112 flipUD pv-train-0000046112
46113 45 pv-train-0000046113
46114 90 pv-train-0000046114
46115 135 pv-train-0000046115
46116 180 pv-train-0000046116
46117 225 pv-train-0000046117
46118 270 pv-train-0000046118
46119 315 pv-train-0000046119
46120 flipUD pv-train-0000046120
46121 45 pv-train-0000046121
46122 90 pv-train-0000046122
461

...

47279 315 pv-train-0000047279
47280 flipLR pv-train-0000047280
47281 45 pv-train-0000047281
47282 90 pv-train-0000047282
47283 135 pv-train-0000047283
47284 180 pv-train-0000047284
47285 225 pv-train-0000047285
47286 270 pv-train-0000047286
47287 315 pv-train-0000047287
47288 flipUD pv-train-0000047288
47289 45 pv-train-0000047289
47290 90 pv-train-0000047290
47291 135 pv-train-0000047291
47292 180 pv-train-0000047292
47293 225 pv-train-0000047293
47294 270 pv-train-0000047294
47295 315 pv-train-0000047295
47296 flipUD pv-train-0000047296
47297 45 pv-train-0000047297
47298 90 pv-train-0000047298
47299 135 pv-train-0000047299
47300 180 pv-train-0000047300
47301 225 pv-train-0000047301
47302 270 pv-train-0000047302
47303 315 pv-train-0000047303
47304 https://raw.githubusercontent.com/RTANC/malaria_dataset/master/Vivax/pv-09/Data20200117-09_Wholeslide_Default_Extended_i58j62.jpg
pv-train-0000047304
47305 90 pv-train-0000047305
47306 180 pv-train-0000047306
47307 270 pv-train-0000047307
47

47515 135 pv-train-0000047515
47516 180 pv-train-0000047516
47517 225 pv-train-0000047517
47518 270 pv-train-0000047518
47519 315 pv-train-0000047519
47520 https://raw.githubusercontent.com/RTANC/malaria_dataset/master/Vivax/pv-09/Data20200117-09_Wholeslide_Default_Extended_i63j61.jpg
pv-train-0000047520
47521 45 pv-train-0000047521
47522 90 pv-train-0000047522
47523 135 pv-train-0000047523
47524 180 pv-train-0000047524
47525 225 pv-train-0000047525
47526 270 pv-train-0000047526
47527 315 pv-train-0000047527
47528 flipLR pv-train-0000047528
47529 45 pv-train-0000047529
47530 90 pv-train-0000047530
47531 135 pv-train-0000047531
47532 180 pv-train-0000047532
47533 225 pv-train-0000047533
47534 270 pv-train-0000047534
47535 315 pv-train-0000047535
47536 flipUD pv-train-0000047536
47537 45 pv-train-0000047537
47538 90 pv-train-0000047538
47539 135 pv-train-0000047539
47540 180 pv-train-0000047540
47541 225 pv-train-0000047541
47542 270 pv-train-0000047542
47543 315 pv-train-0000047543
4754

47751 315 pv-train-0000047751
47752 https://raw.githubusercontent.com/RTANC/malaria_dataset/master/Vivax/pv-09/Data20200117-09_Wholeslide_Default_Extended_i67j56.jpg
pv-train-0000047752
47753 45 pv-train-0000047753
47754 90 pv-train-0000047754
47755 135 pv-train-0000047755
47756 180 pv-train-0000047756
47757 225 pv-train-0000047757
47758 270 pv-train-0000047758
47759 315 pv-train-0000047759
47760 flipLR pv-train-0000047760
47761 45 pv-train-0000047761
47762 90 pv-train-0000047762
47763 135 pv-train-0000047763
47764 180 pv-train-0000047764
47765 225 pv-train-0000047765
47766 270 pv-train-0000047766
47767 315 pv-train-0000047767
47768 flipUD pv-train-0000047768
47769 45 pv-train-0000047769
47770 90 pv-train-0000047770
47771 135 pv-train-0000047771
47772 180 pv-train-0000047772
47773 225 pv-train-0000047773
47774 270 pv-train-0000047774
47775 315 pv-train-0000047775
47776 flipUD pv-train-0000047776
47777 45 pv-train-0000047777
47778 90 pv-train-0000047778
47779 135 pv-train-0000047779
477

47994 90 pv-train-0000047994
47995 135 pv-train-0000047995
47996 180 pv-train-0000047996
47997 225 pv-train-0000047997
47998 270 pv-train-0000047998
47999 315 pv-train-0000047999
48000 https://raw.githubusercontent.com/RTANC/malaria_dataset/master/Vivax/pv-09/Data20200117-09_Wholeslide_Default_Extended_i68j60.jpg
pv-train-0000048000
48001 45 pv-train-0000048001
48002 90 pv-train-0000048002
48003 135 pv-train-0000048003
48004 180 pv-train-0000048004
48005 225 pv-train-0000048005
48006 270 pv-train-0000048006
48007 315 pv-train-0000048007
48008 flipLR pv-train-0000048008
48009 45 pv-train-0000048009
48010 90 pv-train-0000048010
48011 135 pv-train-0000048011
48012 180 pv-train-0000048012
48013 225 pv-train-0000048013
48014 270 pv-train-0000048014
48015 315 pv-train-0000048015
48016 flipUD pv-train-0000048016
48017 45 pv-train-0000048017
48018 90 pv-train-0000048018
48019 135 pv-train-0000048019
48020 180 pv-train-0000048020
48021 225 pv-train-0000048021
48022 270 pv-train-0000048022
48023

48230 270 pv-train-0000048230
48231 315 pv-train-0000048231
48232 https://raw.githubusercontent.com/RTANC/malaria_dataset/master/Vivax/pv-09/Data20200117-09_Wholeslide_Default_Extended_i75j58.jpg
pv-train-0000048232
48233 45 pv-train-0000048233
48234 90 pv-train-0000048234
48235 180 pv-train-0000048235
48236 225 pv-train-0000048236
48237 270 pv-train-0000048237
48238 flipLR pv-train-0000048238
48239 90 pv-train-0000048239
48240 135 pv-train-0000048240
48241 180 pv-train-0000048241
48242 270 pv-train-0000048242
48243 315 pv-train-0000048243
48244 flipUD pv-train-0000048244
48245 45 pv-train-0000048245
48246 90 pv-train-0000048246
48247 180 pv-train-0000048247
48248 225 pv-train-0000048248
48249 270 pv-train-0000048249
48250 flipUD pv-train-0000048250
48251 90 pv-train-0000048251
48252 135 pv-train-0000048252
48253 180 pv-train-0000048253
48254 270 pv-train-0000048254
48255 315 pv-train-0000048255
48256 https://raw.githubusercontent.com/RTANC/malaria_dataset/master/Vivax/pv-09/Data202001

In [10]:
print(phase_count["train"])
print(phase_count["test"])

48328
605


In [11]:
# get images from pv_test (Only test)
labels = os.path.join('pv_test.json')

phase = "test"
# degs = [45,90,135,180,225,270,315]

with open(labels,'r') as json_file:
    data = json.load(json_file)
    images = data["_via_img_metadata"]
    for key in images:
        url = images[key]["filename"]
        leng = len(images[key]["regions"])
        bboxs = ""
        if leng > 0:
            name = "pv-" + phase + "-{:010d}".format(phase_count[phase])
            print("{} {}".format(count,url))
            print(name)
            response = requests.get(url)
            img = Image.open(BytesIO(response.content))
            w, h = img.size
            img.save(os.path.join('../data/custom/images',name+".jpg"))
            f = open(os.path.join('../data/custom/labels',name+".txt"),'w')
            rects = []
            for box in images[key]["regions"]:
                b = box["shape_attributes"]
                x1 = b["x"]/w
                y1 = b["y"]/h
                bw = b["width"]/w
                bh = b["height"]/h
                xc = x1 + (bw/2)
                yc = y1 + (bh/2)
                lbl = box["region_attributes"]["type"]
                if lbl == "1":
                    lbl = "1"
                    bboxs += "\n"+ lbl + " " + str(xc) + " " + str(yc) + " " + str(bw) + " " + str(bh)
                    rects.append(BoundingBox(x1=b["x"],y1=b["y"],x2=(b["x"] + b["width"]),y2=(b["y"] + b["height"]),label=lbl))
                elif lbl == "0":
                    lbl = "0"
                    bboxs += "\n"+ lbl + " " + str(xc) + " " + str(yc) + " " + str(bw) + " " + str(bh)
            bboxs = bboxs[1:]
            f.write(bboxs)
            f.close()
            img = np.array(img)
            bbs = BoundingBoxesOnImage(rects,shape=img.shape)
            count += 1
            phase_count[phase] += 1

1329 https://raw.githubusercontent.com/RTANC/malaria_dataset/master/Vivax/pv_test/1.jpg
pv-test-0000000605
1330 https://raw.githubusercontent.com/RTANC/malaria_dataset/master/Vivax/pv_test/10.jpg
pv-test-0000000606
1331 https://raw.githubusercontent.com/RTANC/malaria_dataset/master/Vivax/pv_test/13.jpg
pv-test-0000000607
1332 https://raw.githubusercontent.com/RTANC/malaria_dataset/master/Vivax/pv_test/17.jpg
pv-test-0000000608
1333 https://raw.githubusercontent.com/RTANC/malaria_dataset/master/Vivax/pv_test/25.jpg
pv-test-0000000609
1334 https://raw.githubusercontent.com/RTANC/malaria_dataset/master/Vivax/pv_test/26.jpg
pv-test-0000000610
1335 https://raw.githubusercontent.com/RTANC/malaria_dataset/master/Vivax/pv_test/30.jpg
pv-test-0000000611
1336 https://raw.githubusercontent.com/RTANC/malaria_dataset/master/Vivax/pv_test/32.jpg
pv-test-0000000612
1337 https://raw.githubusercontent.com/RTANC/malaria_dataset/master/Vivax/pv_test/33.jpg
pv-test-0000000613
1338 https://raw.githubuserco